In [31]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
import math
import numpy as np
from qiskit import *
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ibmqfactory.load_account:WARNING:2020-06-30 22:36:45,608: Credentials are already in use. The existing account in the session will be replaced.


In [130]:
def carry2(circuit, qa, qb, qc, start_index):
    circuit.ccx(qa[start_index], qb[start_index], qc[start_index + 1])
    circuit.cx(qa[start_index], qb[start_index]) 
    circuit.ccx(qc[start_index], qb[start_index], qc[start_index + 1]) 

In [131]:
def carrydg2(circuit, qa, qb, qc, start_index):
    circuit.ccx(qc[start_index], qb[start_index], qc[start_index + 1]) 
    circuit.cx(qa[start_index], qb[start_index]) 
    circuit.ccx(qa[start_index], qb[start_index], qc[start_index + 1])

In [132]:
def qrsum2(circuit, qa, qb, qc, start_index):
    circuit.cx(qa[start_index], qb[start_index])
    circuit.cx(qc[start_index], qb[start_index])

In [133]:
def make_adder2(circuit, qa, qb, qc, n):
        
    # top to bottom
    for i in range(0, n, 1):
        carry2(circuit, qa, qb, qc, i)
        circuit.barrier()
    
    circuit.cx(qa[n - 1], qb[n - 1])
    qrsum2(circuit, qa, qb, qc, n - 1)
    circuit.barrier()
    
    # bottom to top
    for i in range(n - 2, -1, -1):
        carrydg2(circuit, qa, qb, qc, i)
        circuit.barrier()
        qrsum2(circuit, qa, qb, qc, i)
        circuit.barrier()

In [134]:
def make_circ(x, y, size):
            
    qa = QuantumRegister(size, "qa")
    qb = QuantumRegister(size, "qb")
    qc = QuantumRegister(size + 1, "qc")
    cr = QuantumRegister(size, "cr")

    circ = QuantumCircuit(qa, qb, qc, cr)
        
    for i in range(len(y)):
        if y[i] == 1:
            circ.x(qa[i])
            
    for i in range(len(x)):
        if x[i] == 1:
            circ.x(qb[i])
    return circ

In [146]:
def get_results(new):
    total = 0
    for i in range(len(new[0]) - 1, -1, -1):
        if new[0][i] == '1':
            total += 2**(len(new[0]) - 1 - i)
    return total

In [147]:
from PIL import Image
image= Image.open('mlk.jpg') 
(width, height) = image.size
pixels = image.load()

for i in range(width):
    for j in range(height):
        x = [int(k) for k in bin(i)[2:]]
        y = [int(l) for l in bin(j)[2:]]
        
        size = 0
        if len(x) > len(y):
            size = len(x)
        else:
            size = len(y) 
        
        #circ = make_circ(x, y, size)
        qa = QuantumRegister(size, "qa")
        qb = QuantumRegister(size, "qb")
        qc = QuantumRegister(size + 1, "qc")
        cr = QuantumRegister(size, "cr")

        circ = QuantumCircuit(qa, qb, qc, cr)
        circ.draw()
        for i in range(len(y)):
            if y[i] == 1:
                circ.x(qa[i])
            
        for i in range(len(x)):
            if x[i] == 1:
                circ.x(qb[i])
        make_adder2(circ, qa, qb, qc, size)
        
        for i in range(size - 2):
            circ.measure(qb[i], cr[i])
        
        
        backend = Aer.get_backend('statevector_simulator')

        shots = 1024
        job =  execute(circ, backend, shots=shots)
        result = job.result()
        print(result)
        counts = result.get_counts()
        
        new = list(result.get_counts().keys())
        new_x = get_results(new)
        print(new_x)
        
        #circ = make_circ(x, y, size)
        qb = QuantumRegister(size, "qb")
        qc = QuantumRegister(size + 1, "qc")
        cr = QuantumRegister(size, "cr")

        circ = QuantumCircuit(qa, qb, qc, cr)
        
        for i in range(len(y)):
            if y[i] == 1:
                circ.x(qa[i])
            
        for i in range(len(x)):
            if x[i] == 1:
                circ.x(qb[i])
        
        make_adder2(circ, qa, qb, qc, size)
        make_adder2(circ, qa, qb, qc, size)
        
        for i in range(size - 2):
            circ.measure(qb[i], cr[i])
        
        backend = provider.get_backend('ibmq_qasm_simulator')

        shots = 1024
        job =  execute(circ, backend, shots=shots)
        result = job.result()
        counts = result.get_counts()
        
        new = list(result.get_counts().keys())
        new_y = get_results(new)
        print(new_y)

Result(backend_name='statevector_simulator', backend_version='0.5.2', date=datetime.datetime(2020, 7, 1, 0, 40, 44, 98987), header=Obj(backend_name='statevector_simulator', backend_version='0.5.2'), job_id='35cbd3d3-6f6b-4f43-8acb-91e025232196', metadata={'max_memory_mb': 8192, 'omp_enabled': True, 'parallel_experiments': 1, 'time_taken': 0.00020541300000000002}, qobj_id='dad3a4e6-a9fc-48af-aae7-6a7869c91b0e', results=[ExperimentResult(data=ExperimentResultData(statevector=array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
       0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j])), header=Obj(clbit_labels=[], creg_sizes=[], memory_slots=0, n_qubits=5, name='circuit4159', qreg_sizes=[['qa', 1], ['qb', 1], ['qc', 2], ['cr', 1]], qubit_labels=[['qa', 0], ['qb', 0], ['qc', 0], ['qc', 1], ['cr', 0]]), meas_level=<Meas

QiskitError: 'No counts for experiment "0"'

In [ ]:
         for i in range(size - 2):
            circ.measure(qb[i], cr[i])
        
        new_x = get_results(circ)
        
        circ = make_circ(x, y, size)
        
        make_adder2(circ, qa, qb, qc, size)
        make_adder2(circ, qa, qb, qc, size)
        
        for i in range(size - 2):
            circ.measure(qb[i], cr[i])
        
        new_y = get_results(circ)

In [78]:
type(bin(4))
ls = [int(i) for i in bin(4)[2:]]
#ls = list(map(int,str(bin(4))))
ls

[1, 0, 0]

In [48]:
def getStatevectorFromImage(path):
    from PIL import Image
    image= Image.open(path)
    (width, height) = image.size
    pixels = image.load()

    ssum = 0
    for i in range(width):
        for j in range(height):
            ssum += pixels[i,j]**2

    m = math.floor(math.log(height, 2))
    n = math.floor(math.log(width, 2))
    stateVector = np.zeros(2**(m+n))
    for i in range(width):
        for j in range(height):
            stateVector[i*height + j] = pixels[i,j]/math.sqrt(ssum)
            
    return stateVector, m, n

In [49]:
def exportQuantumImage(counts, shots, height, width):
    from PIL import Image

    r = math.floor(math.log(width, 2))
    img= Image.new("L", (width, height))
    pixels = img.load()

    maxAmplitude = 256
    medAmplitude = maxAmplitude/2
    med = shots/(height * width)
    for key in counts:
        i = int(key[0:r], 2)
        j = int(key[r:], 2)
        val = round((((counts[key] - med)/med)*medAmplitude) + medAmplitude)
        pixels[i,j] = (val)
        
    return img

In [150]:
stateVector, m, n = getStatevectorFromImage('mlk.jpg')
state = [complex(x) for x in stateVector]
indexes = range(2*n)

columnReg = QuantumRegister(2**n)
rowReg = QuantumRegister(2**n)
cReg = QuantumRegister(2**n)
classicReg = ClassicalRegister(2**n)

circ = QuantumCircuit(rowReg, columnReg, cReg, classicReg)
circ.initialize(state, indexes)

make_adder2(circ, rowReg, columnReg, cReg, n)

backend = provider.get_backend('ibmq_qasm_simulator')

shots = 2024
job =  execute(circ, backend, shots=shots)
result = job.result()
counts = result.get_counts(circ)

In [66]:
exportQuantumImage(counts, 2040, 2**m, 2**n)

In [149]:
job.error_message()

'The number of qubits in the Qobj (96) is higher than the number of qubits supported by the device (32). Error code: 1109.'